# 1. Sorri not veri gud in inglish

Have you ever googled someone's name without knowing exactly how it should be written? Were you ever reluctant to look up the correct spelling of a query you typed? Or just unable to type properly because of being in a rush? Modern search engines usually do a pretty good job in deciphering defective user input. In order to be able to do that, a good spell-checking mechanism should be incorporated into a search procedure. Today we will take one step further towards building a good search engine and work on tolerant retrieval with respect to user queries. We will consider two cases:

1. Users know that they don't know the correct spelling OR they want to get the results that follow some known pattern, so the use so called wildcards - queries like `retr*val`;
2. Users don't know the correct spelling OR they don't care OR they are in a rush OR they expect that mistakes will be corrected OR /your option/... so they make mistakes and we need to handle them using:

    2.1. Trigrams with Jaccard coefficient;
    
    2.2. Simple spellchecker by Peter Norvig with QWERTY weights;

## 1.1. Handling wildcards

We will handle wildcard queries using k-grams. K-gram is a list of consecutive k chars in a string - i.e., for the word *'star'*, it will be '*\$st*', '*sta*', '*tar*', and '*ar$*', if we take k=3. Take a look at the [book](https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf)'s *chapter 3.2.2* to understand how k-grams can help efficiently match a wildcard against dictionary words. Here we will only consider wildcards with star symbols (may be multiple).

Notice that for building k-grams index, **we will need a vocabulary of original (correct) word forms** to compare words in user input to the vocabulary of "correct" words (think why inverted index which we built for stemmed words doesn't work here).   

You need to implement the following:

- `build_inverted_index_orig_forms` - creates inverted index of original world forms from `facts` list, which is already given to you.  
    Output format: `term:[collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]`
    

- `build_k_gram_index` - creates k-gram index which maps every k-gram encountered in facts collection to a list of words containing this k-gram. Use the abovementioned inverted index of original words to construct this index.  
    Output format: `'k_gram': ['word1_with_k_gram', 'word2_with_k_gram', ...]`
    
    
- `generate_wildcard_options` - produce a list of vocabulary words matching given wildcard by intersecting postings of k-grams present in the wildcard (refer to *ch 3.2.2*). 

- `search_wildcard` - return list of facts that contain the words matching a wildcard query.


We will use the dataset with curious facts for testing.

### 1.1.1. Downloading the dataset

In [4]:
import urllib.request

import nltk

data_url = "https://raw.githubusercontent.com/IUCVLab/information-retrieval/main/datasets/facts.txt"
local_filename, headers = urllib.request.urlretrieve(data_url)

facts = []
with open(local_filename) as fp:
    for cnt, line in enumerate(fp):
        facts.append(line.strip('\n'))

print(*facts[-5:], sep='\n')

151. Women have twice as many pain receptors on their body than men. But a much higher pain tolerance.
152. There are more stars in space than there are grains of sand on every beach in the world.
153. For every human on Earth there are 1.6 million ants.
154. The total weight of all those ants, however, is about the same as all the humans.
155. On Jupiter and Saturn it rains diamonds.


### 1.1.2. Implementation of search

In [14]:
import re
from nltk.corpus import stopwords
from collections import Counter


def is_apt_word(word):
    return word.isalpha() and word not in stopwords.words()


def build_inverted_index_orig_forms(documents):
    #TODO build an inverted index of original word forms 
    # (without stemming, just word tokenized and lowercased)   
    inverted_index = {}
    tokenized_docs = [[word for word in nltk.word_tokenize(document.lower()) if is_apt_word(word)] for document in
                      documents]
    for i, document in enumerate(tokenized_docs):
        document_index = Counter(document)
        for term in document_index.keys():
            if term in inverted_index:
                inverted_index[term].append((i, document_index[term]))
                inverted_index[term][0] += document_index[term]
            else:
                inverted_index[term] = [document_index[term], (i, document_index[term])]

    return inverted_index


def get_k_grams(word, k):
    return [word[i:(i + k)] for i in range(len(word) - k)]


def build_k_gram_index(inverted_index, k):
    #TODO build index of k-grams for dictionary words. 
    # Pad with '$' ($word$) before splitting to k-grams  
    k_gram_index = {}
    for word in inverted_index.keys():
        word = f'${word}$'
        word_k_grams = get_k_grams(word, k)
        for k_gram in word_k_grams:
            if k_gram in k_gram_index:
                k_gram_index[k_gram].add(word)
            else:
                k_gram_index[k_gram] = set(word)
    return k_gram_index


# $1212*123$ -> $1 12 21 12 2* *1 12 23 3$
# $wow*owo$ -> $w wo ow wo ow wo w$
# $wtupwowaaaaaaw$

def matches_wildcard(wildcard, word):
    # Split the wildcard by the '*' character
    parts = wildcard.split('*')

    # Iterate over the split parts
    for part in parts:
        if part:
            # Check if the part is in the word
            idx = word.find(part)
            if idx == -1:
                return False
            # Remove the matched part and continue checking the rest
            word = word[idx + len(part):]

    # If we have checked all parts, the word matches the wildcard
    return True


def generate_wildcard_options(wildcard, k_gram_index, inverted_index):
    #TODO for a given wildcard return all words matching it using k-grams
    # refer to book chapter 3.2.2
    # don't forget to pad wildcard with '$', when appropriate  
    k = len(k_gram_index.keys()[0])
    wildcard = f'${wildcard}$'
    wildcard_k_grams = get_k_grams(wildcard, k)
    words_set = set.intersection(
        *[k_gram_index[k_gram] for k_gram in wildcard_k_grams if k_gram in k_gram_index.keys()])
    return [matches_wildcard(wildcard, word) for word in words_set]


def search_wildcard(wildcard, k_gram_index, index, docs):
    #TODO retrive list of documnets (facts) that contain words matching wildcard

    return []

### 1.1.3. Tests

In [15]:
index_orig_forms = build_inverted_index_orig_forms(facts)
k_gram_index = build_k_gram_index(index_orig_forms, 3)

wildcard = "re*ed"

wildcard_options = generate_wildcard_options(wildcard, k_gram_index, index_orig_forms)
print(wildcard_options)
assert (len(wildcard_options) >= 3)
assert ("red" not in wildcard_options)

wildcard_results = search_wildcard(wildcard, k_gram_index, index_orig_forms, facts)
# some pretty printing
for r in wildcard_results:
    # highlight terms for visual evaluation
    for term in wildcard_options:
        r = re.sub(r'(' + term + ')', r'\033[1m\033[91m\1\033[0m', r, flags=re.I)
    print(r)

assert (len(wildcard_results) >= 3)

assert "13. James Buchanan, the 15th U.S. president continuously bought slaves with his own money in order to free them." in search_wildcard(
    "pres*dent", k_gram_index, index_orig_forms, facts)
assert "40. 9 out of 10 Americans are deficient in Potassium." in search_wildcard("p*tas*um", k_gram_index,
                                                                                  index_orig_forms, facts)
assert "61. A man from Britain changed his name to Tim Pppppppppprice to make it harder for telemarketers to pronounce." in search_wildcard(
    "*price", k_gram_index, index_orig_forms, facts)

TypeError: 'dict_keys' object is not subscriptable

## 1.2. Handling typos

### 1.2.0. Dataset 

Download github typo dataset from [here](https://github.com/mhagiwara/github-typo-corpus).
Load it with this code:

In [ ]:
!pip install jsonlines

In [ ]:
import jsonlines

dataset_file = "github-typo-corpus.v1.0.0.jsonl"

dataset = []
other_langs = set()

with jsonlines.open(dataset_file) as reader:
    for obj in reader:
        for edit in obj['edits']:
            if edit['src']['lang'] != 'eng':
                other_langs.add(edit['src']['lang'])
                continue

            if edit['is_typo']:
                src, tgt = edit['src']['text'], edit['tgt']['text']
                if src.lower() != tgt.lower():
                    dataset.append((edit['src']['text'], edit['tgt']['text']))

print(f"Dataset size = {len(dataset)}")

#### Explore sample typos
Please, explore the dataset. You may see, that this is
- mostly markdown
- some common mistakes with do/does
- some just refer to punctuation typos (which we do not consider)

In [ ]:
for pair in dataset[1010:1020]:
    print(f"{pair[0]} => {pair[1]}")

#### 1.2.0.1. Build a dataset vocabulary
Here we prepare a vocabulary for spellchecker testing and for estimating overall correction quality. Consider only word-level. Be carefull, there is markdown (e.g. \`name\`. \[url\]\(http://url)) and comment symbols (\#, //, \*).

In [ ]:
def sent_to_words(sent):
    # splits sentence to words, filtering out non-alphabetical terms
    words = nltk.word_tokenize(sent)
    words_filtered = filter(lambda x: x.isalpha(), words)
    return words_filtered

In [ ]:
from collections import Counter

vocabulary = Counter()
for pair in dataset:
    for word in sent_to_words(pair[1].lower()):
        vocabulary[word] += 1
len(vocabulary)

In [ ]:
from itertools import islice

print(list(islice(vocabulary.items(), 10)))

### 1.2.1. Implement context-independent spellcheker (Trigrams with Jaccard coefficient) ##

In [ ]:
def fix_typo_kgram(word, k_gram_index) -> list:
    #TODO return best matches with respect to Jaccard index   

    return []

### 1.2.2. Tests

In [ ]:
# tests

k_gram_index_github = build_k_gram_index(vocabulary, 3)
print(fix_typo_kgram("enouh", k_gram_index_github)[:20])
assert "enough" in fix_typo_kgram("enouh", k_gram_index_github), "Assert k-gram failed"

## 1.3. [Extra tasks, for fun]

### 1.3.1. QWERTY - Editorial distance

Write the code to compute weighted QWERTY-editorial distance.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/KB_United_States.svg/640px-KB_United_States.svg.png" width="640"/> 

Use this image to prepare weight function:
- all letter pairs which share vertical border will get 0.5 multiplier **for replace** (`df`, `cv`, `ui`, ...)
- all letter pairs which share at least some horizontal border will get 0.7 multiplier **for replace** (`dc`, `dr`, `km`, ...)
- other operations are not scaled (x1 multiplier).

In [ ]:
def replace_weight(let1, let2) -> float:
    # TODO what is the weight for a pair of letters being replaced? 
    # Note this function should be commutative

    return 0.0


def qwerty_edit_dist(s1, s2) -> float:
    # TODO compute the Damerau-Levenshtein distance 
    # between two given strings (s1 and s2)

    return 0.0

In [ ]:
# tests

assert qwerty_edit_dist("korrectud", "corrected") == 2.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("soem", "some") == 1.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("one", "one") == 0.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("fihure", "figure") == 0.5, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("fivure", "figure") == 0.7, "Edit distance is computed incorrectly"

### 1.3.2. Norvig's spellchecker with QWERTY weights

You can base your code on [Norvig's corrector](https://norvig.com/spell-correct.html), but you should be sure you account the fact, that typos for close letters cost less. This should be considered in ranking.

In [ ]:
def fix_typo_qwerty_norvig(word) -> str:
    #TODO return best matching result for the word

    return ""

In [ ]:
# tests

assert fix_typo_qwerty_norvig("korrectud") == "corrected", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("speling") == "spelling", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("condidence") == "confidence", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("fpx") == "fox", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("fux") == "fix", "Norvig's correcter doesn't work"

### 12.3.3. Estimate quality of functions

In [ ]:
norvig, kgram = 0, 0
limit = 10000
counter = limit
for i, (src, target) in enumerate(dataset):
    if i == limit:
        break
    words = sent_to_words(src.lower())
    # word suspected for typos
    sn, sk = src.lower(), src.lower()
    for word in words:
        if word not in vocabulary and word.isalpha():
            # top-1 accuracy
            wn, wk = fix_typo_qwerty_norvig(word), \
                fix_typo_kgram(word, k_gram_index_github)[0]
            sn = sn.replace(word, wn)
            sk = sk.replace(word, wk)
    norvig += int(sn == target.lower())
    kgram += int(sk == target.lower())

print(f"Norvig accuracy ({norvig}) = {norvig / limit}")
print(f"k-gram accuracy ({kgram}) = {kgram / limit}")